# **Binomial Options Pricing Model** <br>
This notebook will describe the derivations to equations in the BOPM followed by the strategy from a developers standpoint for implementation. The model will be implemented in both a slow and fast way.

## **Binomial Model Derivation - Replicating Portfolio**
The approach I will be taking to develop this model will be via the **Replicating Portfolio Approach**.

- Here the perspective is from the seller of the European Call Option. 
- Once the option has been sold, the seller is in a short position and is exposed to potential financial liability. 
- Therefore the seller decides to hedge this short position by buying stocks (and investing in risk-free bonds). 
- If the portfolio replicates the payoff of the option at each node, the seller is considered perfectly hedged.

### **Portfolio** 

We are going to formulate this model for a one-period binomial tree i.e. the option expires after one time period.

Let $\Delta$ be the number of shares of stock bought
Let $B$ be the amount of money invested in risk-free bonds

The value of the portfolio $C$ is given by $$C = \Delta \cdot S_0 + B$$
where $S_0$ is the initial stock price when the option was sold

Now to be completely hedged, the portfolio needs to match the option's payoff at that time depending on if the stock goes up or down.

If the stock goes up: $$\Delta \cdot S_u + Be^{r \Delta t} = C_u$$

If the stock goes down: $$\Delta \cdot S_d + Be^{r \Delta t} = C_d $$

where $S_u = S_0 \cdot u$ , $S_d = S_0 \cdot d$ , $u$ is the up-factor of the stock and $d = \frac{1}{u}$

Note that the risk free interest rate applied is the same regardless of stock movement since it is a different market.

### **Solving $\Delta$ and $B$**

Subtracting both equations we end up with the formula for $\Delta$: $$\Delta = \frac{C_u - C_d}{S_u - S_d}$$

Substituing this into either the stock up/down equation we obtain $$B = e^{-r \Delta t} \left( \frac{S_u C_d - S_d C_u}{S_u - S_d} \right)$$

### **Calculating the Initial/Previous Option Price**

Note that the previous option price is given by the portfolio value at that time: $$C = \Delta \cdot S_0 + B$$
We can substitue $\Delta$ and $B$ into this to get: $$C = \frac{C_u - C_d}{S_u - S_d} + e^{-r \Delta t} \left( \frac{S_u C_d - S_d C_u}{S_u - S_d} \right)$$ 

This after simplifcation and substituion with variables expressed in terms of $u$ and $d$ results in the **Discounted Expectation Formula**. 

$$C = e^{-r \Delta t} [pC_u + (1-p)C_d]$$ 

where $$ p = \frac{e^{r \Delta t - d}}{u - d}$$ and $$ u = e^{\sigma \sqrt{\Delta t}} , d = \frac{1}{u}$$

This could have been calculated with a simpler probablistic approach of expected value but the approach above demonstrates how the option price is derived using a hedging portfolio to eliminate risk and still acheive arbitrage-free pricing.

## **Binomial Tree - Variables, Equations** - European Call<br>
If we consider the binomial tree to be nodes that can be represented by vectors we can say:

Stock Price at Node i,j: $$S_i,_j = S_0 \cdot u^j \cdot d^{i-j}$$

Consider a European call option. The option contract price is given by $C_{i,j}$. For a given node, the options contract price at that node is given by
$$C_{i,j} = max(S_{i,j} - K, 0)$$

To hedge the short position, buy $\beta$ shares of stock. As a result, consider $\alpha$ amount of money from the monkey market with risk free interest rate r. $\alpha$ is hence given by $$ \alpha = C_0 - \beta S_0$$

Because we are considering a one period binomial model, we consider 2 cases:

(1) The stock goes up. This leads to this formula: $$C_u = \alpha \cdot e^{rT} + \beta \cdot S_u$$
(2) The stock goes down. This leads to this formula: $$C_d = \alpha \cdot e^{rT} + \beta \cdot S_d$$

Since we want to neutralise our risk, we want to buy $\beta$ shares of stock such that we do not lose money regardless of if the stock goes up or down. So we have to say that the portfolio value of our hedge must be equal for both scenarios.

Hence subtracting both equations, we get that we should buy $\beta$ shares given by: $$\beta = \frac{C_u - C_d}{S_u - S_d}$$

By substituting $\beta$ into either equation (1) or (2) we get the arbitrage price for the European call option with one period: $$ C_0 = e^{-rT}[p C_u + q C_d] $$ 
where $p = \frac{e^{rT} - d}{u - d}$ and $q = 1 - p$ which is the risk-neutral probability.

NB: $S_u = S_0 \cdot u$ and $S_d = S_0 \cdot d$

In [9]:
import numpy as np

In [10]:
# defining variables
S0 = 100    # initial stock price
K = 103     # strike price
T = 1       # time to maturity (years)
N = 3       # no. of time-steps
r = 0.06    # risk-free annual interest rate
u = 1.2     # up factor
d = 1/u     # down factor (inverse for recombining tree)
opt = 'C'   # options type (C - call ; P - put)

## **Binomial Tree - for loops (Slow)**
In this model, we will iterate through the notes using for loops which is slow. Later we will see how to speed this up using numpy vectorisation. 

The strategy is to first compute the contract values at the terminal node layer and work backwards till the initial contract value which will give us the fair options price.

In [11]:
def binomial_slow(S0,K,T,N,r,u,d,opt='C'):
    # first compute equation constants
    dt = T/N 
    p = (np.exp(r*dt) - d)/(u - d)
    discount = np.exp(-r * dt)

    # numpy array to store the stock values at maturity
    S = np.zeros(N+1)
    S[0] = d**N * S0
    for j in range(1,N+1):
        S[j] = S0 * (u ** j) * (d ** (N-j))

    # numpy array to store options values at maturity'
    C = np.zeros(N+1)
    for j in range(0,N+1):
        C[j] = max(0,S[j] - K)
    
    # now let's move backwards through the tree
    for i in np.arange(N,0,-1):
        for j in range(0,i):
            # replace the cell in the nparray with the backwards contract calculation of itself and the one above
            # essentially knocking off the top element            
            C[j] = discount * (p * C[j+1] + (1-p) * C[j]) 
    
    return C[0]

In [12]:
result = round(float(binomial_slow(S0,K,T,N,r,u,d,opt='C')),2)
result

14.82

## **Binomial Tree - numpy vectorisation (Fast)**

The same mathematics and techniques are going to be used here but instead of iterating using for loops we will speed up the multiplication process by using vectors.

The strategy to be used is as follows:
1) We initialise a numpy array (vector) **C** using np.arange(). This will contain the stock prices at the terminal node layer

2) Then we subtract the scalar **K** from the vector **C** and compute the maximum for each entry in the vector between $ K - C $ and the zero vector. This will satisfy this equation $$C_{i,j} = max(S_{i,j} - K, 0)$$

3) Now we step backwards through the tree from the Nth (terminal) layer to the 1st layer (not the 0th layer). In order to efficiently calculate the previous layer's contract prices we will use vector slicing. To calculate the $i-1$ node at a height $j$, we take the nodes from $i$ at positions $j$ and $j+1$ and we do this for every single node at layer $i-1$. For each node this acts as a one period binomial tree. It therefore makes sense to slice the $i$ th layer's vector from $0$ to $i-1$ (inclusive) for the node positions at $j$ and slice the $i$ th layer's vector from $1$ to $i$ (inclusive) for node positions at $j+1$ for each one period binomial tree. 

4) Once we've reached the final one period binomial tree, the initial node will have been calculated (i.e. its contract price has been calculated) so we output that



In [13]:
def binomial_fast(S0=100,K=103,T=1,N=3,r=0.06,u=1.2,d=1/u,opt='C'):
    # first compute equation constants
    dt = T/N 
    p = (np.exp(r*dt) - d)/(u - d)
    discount = np.exp(-r * dt)

    #initialise array with stock prices
    C = S0 * u ** (np.arange(0,N+1,1)) * d ** (np.arange(N,-1,-1))

    #calculate option payoffs using vectors
    C = np.maximum(C-K, np.zeros(N+1))

    #iterate backwards and multiply vectors so that each index in previous step corresponds
    #to the same index times the one above it - hence the slicing
    for i in np.arange(N,0,-1):
        C = discount * (p * C[1:i+1] + (1-p) * C[0:i])
    
    return C[0]

In [14]:
result = round(float(binomial_fast(S0=100,K=103,T=1,N=3,r=0.06,u=1.2,d=1/u,opt='C')), 2)
result

14.82